## Importar das Bibliotecas

In [ ]:
import pandas as pd
import multiprocessing as mp
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
nome = '' #nome do local
ano = #ano a analisar

In [ ]:
Grafico = pd.read_excel(f'..\\data\\Reconstruídas\\{nome.replace(' ','_')}_{ano}.xlsx')
Grafico_Outliers = pd.read_excel(f'..\\data\\Resultados\\{nome.replace(' ','_')}_{ano}_Resultado.xlsx')

In [ ]:
# Melt the dataframe to convert it to two columns: 'Date' and 'Value'
Grafico_Derretido = Grafico.melt(id_vars=['Date'], var_name='Hora', value_name='Caudal')
Grafico_Outliers_Derretido = Grafico_Outliers.melt(id_vars=['Date'], var_name='Hora', value_name='Anomalia')

#Porto_Carro_2016_Grafico_Derretido

In [ ]:
# Converter a coluna 'Date' para datetime
Grafico_Derretido['Date'] = pd.to_datetime(Grafico_Derretido['Date'], format='%d/%m/%Y', dayfirst=True)

# Adicionar uma coluna de mês
Grafico_Derretido['Mês'] = Grafico_Derretido['Date'].dt.month

# Adicionar uma coluna de ano
Grafico_Derretido['Ano'] = Grafico_Derretido['Date'].dt.year

In [ ]:
# Converter a coluna 'Date' para datetime
Grafico_Outliers_Derretido['Date'] = pd.to_datetime(Grafico_Outliers_Derretido['Date'], format='%d/%m/%Y', dayfirst=True)

# Adicionar uma coluna de mês
Grafico_Outliers_Derretido['Mês'] = Grafico_Outliers_Derretido['Date'].dt.month

# Adicionar uma coluna de ano
Grafico_Outliers_Derretido['Ano'] = Grafico_Outliers_Derretido['Date'].dt.year

# Adicionar uma coluna de Caudal a tabela de Outliers
Grafico_Outliers_Derretido['Caudal'] = Grafico_Derretido['Caudal']

In [ ]:
# Group by 'Mês' and 'Hora' and calculate the required statistics
statistics = Grafico_Derretido.groupby(['Mês', 'Hora', 'Ano']).agg(
    Percentil_95=('Caudal', lambda x: np.percentile(x, 95)),
    Media=('Caudal', 'mean'),
    Mediana=('Caudal', 'median'),
    Percentil_5=('Caudal', lambda x: np.percentile(x, 5))
).reset_index()

#statistics

In [ ]:
# Create a boxplot for the 'Caudal' column in the Porto_Carro_2016_Grafico_Derretido dataframe
#Grafico_Derretido.boxplot(column='Caudal', by='Mês', figsize=(50, 15))
Grafico_Derretido.boxplot(column='Caudal', by='Mês', figsize=(25, 5))
plt.title(f'Boxplot do Caudal por Mês no Ano {ano} em {nome}')
plt.title(f' ')
plt.suptitle('')  # Suppress the default title to avoid overlap

# Defenir a cor de fundo e transparência
plt.gca().set_facecolor('none')  # Set the axes background to be transparent
plt.gcf().set_facecolor('none')  # Set the figure background to be transparent
plt.gcf().set_alpha(0)  # Set the figure background to be transparent

plt.xlabel('Mês')
plt.ylabel('Caudal')

#Criar pasta para guardar
os.makedirs(f'..\\data\\Gráficos\\Boxplot\\{nome}', exist_ok=True)

#Gravar gráficos
plt.savefig(f'..\\data\\Gráficos\\Boxplot\\{nome}\\Boxplot - {nome} - {ano}.svg', dpi=10000)
plt.savefig(f'..\\data\\Gráficos\\Boxplot\\{nome}\\Boxplot - {nome} - {ano}.png', dpi=250)
plt.savefig(f'..\\data\\Gráficos\\Boxplot\\{nome}\\Boxplot - {nome} - {ano}.tiff', dpi=250)

plt.show()

# Describe the 'Caudal' column by 'Mês'
Grafico_Derretido.groupby('Mês')['Caudal'].describe()

In [ ]:
for mes in Grafico_Derretido['Mês'].unique():
    # Ensure the data is sorted by 'Date' and 'Hora'
    Grafico_Derretido_Tudo = Grafico_Derretido.sort_values(by=['Date', 'Hora'])
    Grafico_Derretido_Tudo_Amomalias_Comunicadas = Grafico_Outliers_Derretido.sort_values(by=['Date', 'Hora'])
    Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers = Grafico_Outliers_Derretido.sort_values(by=['Date', 'Hora'])
    Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers_Menos = Grafico_Outliers_Derretido.sort_values(by=['Date', 'Hora'])
    
    boxplot = Grafico_Derretido.groupby('Mês')['Caudal'].describe()
    
    #
    Grafico_Derretido_Tudo = Grafico_Derretido_Tudo[Grafico_Derretido_Tudo['Mês'] == mes]
    Grafico_Derretido_Tudo_Amomalias_Comunicadas = Grafico_Derretido_Tudo_Amomalias_Comunicadas[Grafico_Derretido_Tudo_Amomalias_Comunicadas['Mês'] == mes]
    Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers = Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers[Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers['Mês'] == mes]
    Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers_Menos = Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers_Menos[Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers_Menos['Mês'] == mes]

    # Convert 'Hora' to a suitable timedelta representation
    Grafico_Derretido_Tudo['Hora'] = Grafico_Derretido_Tudo['Hora'].str.replace('X', '').str.replace('.', ':')
    Grafico_Derretido_Tudo['Hora'] = pd.to_timedelta(Grafico_Derretido_Tudo['Hora'] + ':00')

    Grafico_Derretido_Tudo_Amomalias_Comunicadas['Hora'] = Grafico_Derretido_Tudo_Amomalias_Comunicadas['Hora'].str.replace('X', '').str.replace('.', ':')
    Grafico_Derretido_Tudo_Amomalias_Comunicadas['Hora'] = pd.to_timedelta(Grafico_Derretido_Tudo_Amomalias_Comunicadas['Hora'] + ':00')
    
    Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers['Hora'] = Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers['Hora'].str.replace('X', '').str.replace('.', ':')
    Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers['Hora'] = pd.to_timedelta(Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers['Hora'] + ':00')
    
    Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers_Menos['Hora'] = Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers_Menos['Hora'].str.replace('X', '').str.replace('.', ':')
    Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers_Menos['Hora'] = pd.to_timedelta(Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers_Menos['Hora'] + ':00')

    # Replace 'Caudal' values with np.nan where 'Anomalia' is 0
    Grafico_Derretido_Tudo_Amomalias_Comunicadas.loc[Grafico_Derretido_Tudo_Amomalias_Comunicadas['Anomalia'] == 0, 'Caudal'] = np.nan
    #Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers.loc[Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers['Anomalia'] == 0, 'Caudal'] = np.nan
    
    #
    amplitudeInterQuartil = boxplot.loc[mes, '75%'] - boxplot.loc[mes, '25%']
    
    Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers.loc[Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers['Caudal'] < (boxplot.loc[mes, '75%'] + 1.5 * amplitudeInterQuartil), 'Caudal'] = np.nan
    Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers_Menos.loc[Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers_Menos['Caudal'] > (boxplot.loc[mes, '25%'] - 1.5 * amplitudeInterQuartil), 'Caudal'] = np.nan

    # Combine 'Date' and 'Hora' into a single datetime column
    Grafico_Derretido_Tudo['DateTime'] = Grafico_Derretido_Tudo['Date'] + Grafico_Derretido_Tudo['Hora']
    Grafico_Derretido_Tudo_Amomalias_Comunicadas['DateTime'] = Grafico_Derretido_Tudo_Amomalias_Comunicadas['Date'] + Grafico_Derretido_Tudo_Amomalias_Comunicadas['Hora']
    Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers['DateTime'] = Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers['Date'] + Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers['Hora']
    Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers_Menos['DateTime'] = Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers_Menos['Date'] + Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers_Menos['Hora']

    # Plot the data
    #plt.figure(figsize=(75, 15))
    plt.figure(figsize=(25, 5))
    plt.plot(Grafico_Derretido_Tudo['DateTime'], Grafico_Derretido_Tudo['Caudal'], label='Caudal', color='xkcd:azure')
    plt.plot(Grafico_Derretido_Tudo_Amomalias_Comunicadas['DateTime'], Grafico_Derretido_Tudo_Amomalias_Comunicadas['Caudal'], label='Caudal - Anomalias Comunicadas', color='xkcd:tomato')
    plt.plot(Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers['DateTime'], Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers['Caudal'], linestyle=" ", marker='x', label='Caudal - Outliers - Acima do Máximo', color="xkcd:orange")
    plt.plot(Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers_Menos['DateTime'], Grafico_Derretido_Tudo_Amomalias_Comunicadas_BoxPlot_Outliers_Menos['Caudal'], linestyle=" ", marker='x', label='Caudal - Outliers - Abaixo do Mínimo', color="xkcd:orange")
    
    plt.axhline(boxplot.loc[mes, '75%'] + 1.5 * amplitudeInterQuartil, color='xkcd:blue', linestyle='--',label='Boxplot - Máximo')
    plt.axhline(boxplot.loc[mes, '25%'] - 1.5 * amplitudeInterQuartil, color='xkcd:blue', linestyle='--',label='Boxplot - Mínimo')
    
    plt.axhline(boxplot['75%'][mes], color='xkcd:purple', linestyle='--', label='Boxplot - 3º Quartil - 75%')
    plt.axhline(boxplot['50%'][mes], color='xkcd:green', linestyle='--', label='Boxplot - 2º Quartil - 50%')
    plt.axhline(boxplot['25%'][mes], color='xkcd:fuchsia', linestyle='--', label='Boxplot - 1º Quartil - 25%')
    
    plt.axhline(boxplot['mean'][mes], color='xkcd:gold', linestyle='--', label='Boxplot - Média')
    plt.axhline(boxplot['std'][mes], color='xkcd:silver', linestyle='--', label='Boxplot - Desvio Padrão')
    
    # Add vertical lines for each day
    for dia in Grafico_Derretido_Tudo['Date'].unique():
        plt.axvline(pd.to_datetime(dia) + pd.to_timedelta('00:00:00'), color='xkcd:black', linestyle='--', alpha=0.5)
    
    plt.xlabel('Dia')
    plt.ylabel('Caudal')
    #plt.title(f'Caudal no {mes}º Mês do Ano {ano} em {nome}')
    
    # Defenir a cor de fundo e transparência
    plt.gca().set_facecolor('none')  # Set the axes background to be transparent
    plt.gcf().set_facecolor('none')  # Set the figure background to be transparent
    plt.gcf().set_alpha(0)  # Set the figure background to be transparent
    
    plt.legend().set_alpha(0)
    plt.grid(True)
    
    # Criar uma pasta para guardar os gráficos
    os.makedirs(f'..\\data\\Gráficos\\Plot de Tudo\\{nome}\\{ano}', exist_ok=True)
    
    # Save the plot in different formats in a different directory with higher resolution
    plt.savefig(f'..\\data\\Gráficos\\Plot de Tudo\\{nome}\\{ano}\\Plot - {nome} - Ano {ano} - Mês {mes}.svg', dpi=10000)
    plt.savefig(f'..\\data\\Gráficos\\Plot de Tudo\\{nome}\\{ano}\\Plot - {nome} - Ano {ano} - Mês {mes}.png', dpi=250)
    plt.savefig(f'..\\data\\Gráficos\\Plot de Tudo\\{nome}\\{ano}\\Plot - {nome} - Ano {ano} - Mês {mes}.tiff', dpi=250)
    
    plt.show()
    
    